In [ ]:
首次到期的用户


In [13]:
import os
import sys
import time
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import re
import math
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
sys.path.append('/Users/yantingting/PycharmProjects/modeling/')
import database_conncet as data_con
import data_processing as dp
import summary_statistics as ss
from data_io_utils import *
from matplotlib.pyplot import MultipleLocator
from jinja2 import Template

In [2]:
file_path= '/Users/yantingting/Seafile/风控/模型/10 印尼/202004 老客决策树/01 Data'

# y

In [24]:
query0 = '''
select t1.*,t2.late_day,
case when t2.late_day>0 then 1 else 0 end as dpd1
from 
(select id as loan_id,effective_date,due_date,approved_period
from dw_gocash_go_cash_loan_gocash_core_loan 
where return_flag = 'true' and due_date::date>='2020-03-20' and due_date::date<='2020-04-07' and due_date - effective_date+1 = approved_period) t1 
left join 
(select *
from dw_gocash_gocash_collection_col_case 
where due_date - approved_date+1 = approved_period and order_status != 'PAIDOFF' and app_id not in ('Credits','KASANDAAI')) t2 
on t1.loan_id = t2.order_id ;
'''

In [25]:
df0 = data_con.database_gn(query0)
save_data_to_pickle(df0,file_path,'y.pkl')
df0.shape
df0.head()

Successful connection


(41107, 6)

,loan_id,effective_date,due_date,approved_period,late_day,dpd1
0,443078651486502912,2020-03-07,2020-04-04,29,3.0,1
1,443080428537413632,2020-03-07,2020-03-28,22,NaN,0
2,443085055811223552,2020-03-07,2020-04-04,29,NaN,0
3,443086227296919552,2020-03-07,2020-03-21,15,17.0,1
4,443092329594003456,2020-03-07,2020-03-28,22,10.0,1


In [37]:
# frame1 = pd.crosstab(df0['due_date'],df0['dpd1'])
# frame1['首逾']  = frame1[1]/frame1.sum(axis = 1)
# frame1

# 基本信息

In [3]:
query1 = '''
select t1.loan_id,
t1.device_approval,
t1.product_id,
case when substring(id_card_no, 11,2)::int >substring(cast(apply_time as varchar),3,2)::int 
then 100 +substring(cast(apply_time as varchar),3,2)::int - substring(id_card_no, 11,2)::int
else substring(cast(apply_time as varchar),3,2)::int - substring(id_card_no, 11,2)::int  end as age,
case when substring(id_card_no, 7,2)::int > 40 then 'female' else 'male' end as gender,
split_part(id_card_address ,',', length(replace(id_card_address,',',',,')) - length(id_card_address) + 1) as province,
industry_involved,
monthly_salary,
marital_status,
education
from 
(select id as loan_id,apply_time,customer_id,device_approval,product_id
from dw_gocash_go_cash_loan_gocash_core_loan 
where return_flag = 'true' and due_date::date>='2020-03-20' and due_date::date<='2020-04-07' and due_date - effective_date+1 = approved_period) t1 
left join 
(select id,	id_card_no,	id_card_address,marital_status,education
from dw_gocash_go_cash_loan_gocash_core_customer
where length(id_card_address)>0) t2 
on t1.customer_id = t2.id
left join dw_gocash_go_cash_loan_gocash_core_cusomer_profession t3 
on t1.customer_id = t3.customer_id;
'''

In [4]:
df1 = data_con.database_gn(query1)
save_data_to_pickle(df1,file_path,'baseinfo.pkl')
df1.shape
df1.head()

Successful connection


(41109, 10)

,loan_id,device_approval,product_id,age,gender,province,industry_involved,monthly_salary,marital_status,education
0,443078651486502912,ANDROID,1,51,male,SULAWESI SELATAN,RETAIL,8500000.0,MARRIED,SENIOR_HIGH_SCHOOL
1,443090442014785536,ANDROID,1,28,male,JAWA TIMUR,OTHER,4700000.0,SPINSTERHOOD,REGULAR_COLLEGE_COURSE
2,443091336617885696,ANDROID,1,32,male,JAWA TENGAH,ARCHITECTURE,5000000.0,MARRIED,JUNIOR_HIGH_SCHOOL
3,443096277470052352,ANDROID,1,38,male,JAWA BARAT,ARCHITECTURE,9000000.0,MARRIED,SENIOR_HIGH_SCHOOL
4,443111701574418432,ANDROID,1,33,male,JAWA TIMUR,INTERNET,0.0,MARRIED,ASSOCIATE_BACHELOR_3


# 历史行为

In [5]:
query2 = '''
SELECT t1.loan_id			
--近N天放款次数
, count(distinct case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and date(t1.apply_time) > date(t2.effective_date) and t2.effective_date != '1970-01-01'
					then t2.id else null end) as cnt_loan30
, count(distinct case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and date(t1.apply_time) > date(t2.effective_date) and t2.effective_date != '1970-01-01'
					then t2.id else null end) as cnt_loan60
, count(distinct case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and date(t1.apply_time) > date(t2.effective_date) and t2.effective_date != '1970-01-01'
					then t2.id else null end) as cnt_loan90
, count(distinct case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and date(t1.apply_time) > date(t2.effective_date) and t2.effective_date != '1970-01-01'
					then t2.id else null end) as cnt_loan180
, count(distinct case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and date(t1.apply_time) > date(t2.effective_date) and t2.effective_date != '1970-01-01'
					then t2.id else null end) as cnt_loan360
, count(distinct case when date(t1.apply_time) > date(t2.effective_date) and t2.effective_date != '1970-01-01'
					then t2.id else null end) as cnt_loan					
--第一次/最近一次申请距离现在申请时间间隔
, max(date(t1.apply_time) - date(t2.apply_time)) as firstapply_curds
, min(date(t1.apply_time) - date(t2.apply_time)) as lastapply_curds
--第一次/最近一次放款距离现在申请时间间隔
, max(case when date(t1.apply_time) > date(t2.effective_date) and t2.effective_date != '1970-01-01' then date(t1.apply_time) - date(t2.effective_date) else null end) as firstloan_curds
, min(case when date(t1.apply_time) > date(t2.effective_date) and t2.effective_date != '1970-01-01' then date(t1.apply_time) - date(t2.effective_date) else null end) as last_loan_curds
--期限
, max(case when t2.effective_date != '1970-01-01' then t2.approved_period end) as  max_loan_period
, max(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.effective_date != '1970-01-01' then t2.approved_period end) as  max_loan_period30d
, max(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.effective_date != '1970-01-01' then t2.approved_period end) as  max_loan_period60d
, max(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.effective_date != '1970-01-01' then t2.approved_period end) as  max_loan_period90d
, max(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.effective_date != '1970-01-01' then t2.approved_period end) as  max_loan_period180d
, max(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.effective_date != '1970-01-01' then t2.approved_period end) as  max_loan_period360d
, min(case when t2.effective_date != '1970-01-01' then t2.approved_period end) as  min_loan_period
, min(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.effective_date != '1970-01-01' then t2.approved_period end) as  min_loan_period30d
, min(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.effective_date != '1970-01-01' then t2.approved_period end) as  min_loan_period60d
, min(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.effective_date != '1970-01-01' then t2.approved_period end) as  min_loan_period90d
, min(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.effective_date != '1970-01-01' then t2.approved_period end) as  min_loan_period180d
, min(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.effective_date != '1970-01-01' then t2.approved_period end) as  min_loan_period360d
, 1.0 * sum(case when t2.effective_date != '1970-01-01' then t2.approved_period end)/count(case when t2.effective_date != '1970-01-01' then t2.id end)  as  avg_loan_period
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.effective_date != '1970-01-01' then t2.id end) > 0 
    then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.effective_date != '1970-01-01' then t2.approved_period end)
        /count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.effective_date != '1970-01-01' then t2.id end) 
    else null end as  avg_loan_period30d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.effective_date != '1970-01-01' then t2.id end) > 0 
    then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.effective_date != '1970-01-01' then t2.approved_period end)
       /count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.effective_date != '1970-01-01' then t2.id end) 
    else null end as  avg_loan_period60d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.effective_date != '1970-01-01' then t2.id end) > 0 
    then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.effective_date != '1970-01-01' then t2.approved_period end)
       /count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.effective_date != '1970-01-01' then t2.id end)
    else null end as  avg_loan_period90d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.effective_date != '1970-01-01' then t2.id end) > 0 
    then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.effective_date != '1970-01-01' then t2.approved_period end)
      /count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.effective_date != '1970-01-01' then t2.id end) 
    else null end as  avg_loan_period180d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.effective_date != '1970-01-01' then t2.id end) > 0 
    then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.effective_date != '1970-01-01' then t2.approved_period end)
      /count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.effective_date != '1970-01-01' then t2.id end) 
    else null end as  avg_loan_period360d
--期限占比
, count(case when t2.approved_period = 15 and t2.effective_date != '1970-01-01' then t2.id else null end) as cnt_loan_period_is15
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.approved_period = 15 and t2.effective_date != '1970-01-01' then t2.id else null end) as cnt_loan_period_is15_30d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.approved_period = 15 and t2.effective_date != '1970-01-01' then t2.id else null end) as cnt_loan_period_is15_60d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.approved_period = 15 and t2.effective_date != '1970-01-01' then t2.id else null end) as cnt_loan_period_is15_90d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.approved_period = 15 and t2.effective_date != '1970-01-01' then t2.id else null end) as cnt_loan_period_is15_180d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.approved_period = 15 and t2.effective_date != '1970-01-01' then t2.id else null end) as cnt_loan_period_is15_360d
, count(case when t2.approved_period = 8 and t2.effective_date != '1970-01-01' then t2.id else null end) as cnt_loan_period_is8
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.approved_period = 8 and t2.effective_date != '1970-01-01' then t2.id else null end) as cnt_loan_period_is8_30d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.approved_period = 8 and t2.effective_date != '1970-01-01' then t2.id else null end) as cnt_loan_period_is8_60d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.approved_period = 8 and t2.effective_date != '1970-01-01' then t2.id else null end) as cnt_loan_period_is8_90d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.approved_period = 8 and t2.effective_date != '1970-01-01' then t2.id else null end) as cnt_loan_period_is8_180d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.approved_period = 8 and t2.effective_date != '1970-01-01' then t2.id else null end) as cnt_loan_period_is8_360d
----占比
, 1.0 * count(case when t2.approved_period = 15 and t2.effective_date != '1970-01-01' then t2.id else null end)/count(case when t2.effective_date != '1970-01-01' then t2.id end) as rate_loan_period_is15
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30  and t2.effective_date != '1970-01-01' then t2.id else null end) > 0   
        then 1.0 * count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.approved_period = 15 and t2.effective_date != '1970-01-01' then t2.id else null end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.effective_date != '1970-01-01' then t2.id else null end)
        else null end as rate_loan_period_is15_30d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.effective_date != '1970-01-01' then t2.id else null end) > 0   
        then 1.0 * count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.approved_period = 15 and t2.effective_date != '1970-01-01' then t2.id else null end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.effective_date != '1970-01-01'then t2.id else null end) 
        else null end as rate_loan_period_is15_60d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.effective_date != '1970-01-01' then t2.id else null end) > 0   
        then 1.0 * count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.approved_period = 15 and t2.effective_date != '1970-01-01' then t2.id else null end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90  and t2.effective_date != '1970-01-01' then t2.id else null end) 
        else null end as rate_loan_period_is15_90d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.effective_date != '1970-01-01' then t2.id else null end) > 0   
        then 1.0 * count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.approved_period = 15 and t2.effective_date != '1970-01-01' then t2.id else null end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180  and t2.effective_date != '1970-01-01' then t2.id else null end) 
        else null end as rate_loan_period_is15_180d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.effective_date != '1970-01-01' then t2.id else null end) > 0   
        then 1.0 * count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.approved_period = 15 and t2.effective_date != '1970-01-01' then t2.id else null end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360  and t2.effective_date != '1970-01-01' then t2.id else null end) 
        else null end as rate_loan_period_is15_360d
, 1.0 * count(case when t2.approved_period = 8 and t2.effective_date != '1970-01-01' then t2.id else null end)/count(case when t2.effective_date != '1970-01-01' then t2.id else null end) as rate_loan_period_is8
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.effective_date != '1970-01-01' then t2.id else null end) > 0 
      then 1.0 * count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.approved_period = 8 and t2.effective_date != '1970-01-01' then t2.id else null end)/
                count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.effective_date != '1970-01-01' then t2.id else null end) 
    else null end as rate_loan_period_is8_30d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.effective_date != '1970-01-01' then t2.id else null end) > 0
      then 1.0 * count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.approved_period = 8 and t2.effective_date != '1970-01-01' then t2.id else null end)/
                count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60  and t2.effective_date != '1970-01-01' then t2.id else null end)
     else null end as rate_loan_period_is8_60d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.effective_date != '1970-01-01' then t2.id else null end) > 0
      then 1.0 * count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.approved_period = 8 and t2.effective_date != '1970-01-01' then t2.id else null end)/
            count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.effective_date != '1970-01-01' then t2.id else null end) 
    else null end as rate_loan_period_is8_90d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.effective_date != '1970-01-01' then t2.id else null end) > 0
      then 1.0 * count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.approved_period = 8 and t2.effective_date != '1970-01-01'  then t2.id else null end)/
            count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180  and t2.effective_date != '1970-01-01' then t2.id else null end) 
    else null end as rate_loan_period_is8_180d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360  and t2.effective_date != '1970-01-01' then t2.id else null end) > 0
      then 1.0 * count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.approved_period = 8 and t2.effective_date != '1970-01-01' then t2.id else null end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360  and t2.effective_date != '1970-01-01' then t2.id else null end) 
    else null end as rate_loan_period_is8_360d
----展期次数
, count(case when t2.extend_times >0 and t2.effective_date != '1970-01-01' then t2.id else null end) as cnt_extend_times
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.extend_times >0 and t2.effective_date != '1970-01-01'  then t2.id else null end) as cnt_extend_times_30d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.extend_times >0 and t2.effective_date != '1970-01-01'  then t2.id else null end) as cnt_extend_times_60d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.extend_times >0 and t2.effective_date != '1970-01-01'  then t2.id else null end) as cnt_extend_times_90d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.extend_times >0 and t2.effective_date != '1970-01-01'  then t2.id else null end) as cnt_extend_times_180d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.extend_times >0 and t2.effective_date != '1970-01-01'  then t2.id else null end) as cnt_extend_times_360d
, max(t2.extend_times) as max_extend_times
, max(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 then t2.extend_times else null end) as max_extend_times_30d
, max(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 then t2.extend_times else null end) as max_extend_times_60d
, max(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 then t2.extend_times else null end) as max_extend_times_90d
, max(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then t2.extend_times else null end) as max_extend_times_180d
, max(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then t2.extend_times else null end) as max_extend_times_360d
, sum(t2.extend_times) as sum_extend_times
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 then t2.extend_times else null end) as sum_extend_times_30d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 then t2.extend_times else null end) as sum_extend_times_60d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 then t2.extend_times else null end) as sum_extend_times_90d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then t2.extend_times else null end) as sum_extend_times_180d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then t2.extend_times else null end) as sum_extend_times_360d
, 1.0 * sum(t2.extend_times)/ count(case when t2.effective_date != '1970-01-01'  then t2.id end) as avg_extend_times
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.effective_date != '1970-01-01' then t2.id else null end) > 0 
    then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 then t2.extend_times else null end)/
    count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.effective_date != '1970-01-01' then t2.id else null end) 
    else null end as avg_extend_times_30d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.effective_date != '1970-01-01' then t2.id else null end) > 0 
    then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 then t2.extend_times else null end)/
    count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.effective_date != '1970-01-01' then t2.id else null end) 
    else null end as avg_extend_times_60d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.effective_date != '1970-01-01' then t2.id else null end) > 0 
    then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 then t2.extend_times else null end)/
    count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.effective_date != '1970-01-01' then t2.id else null end) 
    else null end as avg_extend_times_90d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.effective_date != '1970-01-01' then t2.id else null end) > 0 
    then 1.0 *  sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then t2.extend_times else null end)/
    count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.effective_date != '1970-01-01' then t2.id else null end) 
    else null end as avg_extend_times_180d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.effective_date != '1970-01-01' then t2.id else null end) > 0 
    then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then t2.extend_times else null end)/
    count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.effective_date != '1970-01-01'then t2.id else null end) 
    else null end as avg_extend_times_360d
----贷后
, count(case when t2.loan_status = 'ADVANCE_PAIDOFF' then t2.id else null end) as cnt_advance_paidoff
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.loan_status = 'ADVANCE_PAIDOFF' then t2.id else null end) as cnt_advance_paidoff30d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.loan_status = 'ADVANCE_PAIDOFF' then t2.id else null end) as cnt_advance_paidoff60d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.loan_status = 'ADVANCE_PAIDOFF' then t2.id else null end) as cnt_advance_paidoff90d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.loan_status = 'ADVANCE_PAIDOFF' then t2.id else null end) as cnt_advance_paidoff180d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.loan_status = 'ADVANCE_PAIDOFF' then t2.id else null end) as cnt_advance_paidoff360d
, count(case when t2.loan_status = 'COLLECTION_PAIDOFF' then t2.id else null end) as cnt_collection_paidoff
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.loan_status = 'COLLECTION_PAIDOFF' then t2.id else null end) as cnt_collection_paidoff30d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.loan_status = 'COLLECTION_PAIDOFF' then t2.id else null end) as cnt_collection_paidoff60d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.loan_status = 'COLLECTION_PAIDOFF' then t2.id else null end) as cnt_collection_paidoff90d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.loan_status = 'COLLECTION_PAIDOFF' then t2.id else null end) as cnt_collection_paidoff180d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.loan_status = 'COLLECTION_PAIDOFF' then t2.id else null end) as cnt_collection_paidoff360d
, count(case when t2.loan_status = 'PAIDOFF' then t2.id else null end) as cnt_paidoff
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and t2.loan_status = 'PAIDOFF' then t2.id else null end) as cnt_paidoff30d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and t2.loan_status = 'PAIDOFF' then t2.id else null end) as cnt_paidoff60d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and t2.loan_status = 'PAIDOFF' then t2.id else null end) as cnt_paidoff90d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and t2.loan_status = 'PAIDOFF' then t2.id else null end) as cnt_paidoff180d
, count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and t2.loan_status = 'PAIDOFF' then t2.id else null end) as cnt_paidoff360d
from (SELECT id as loan_id, apply_time, effective_date, customer_id
      FROM dw_gocash_go_cash_loan_gocash_core_loan
      where return_flag = 'true' and due_date::date>='2020-03-20' and due_date::date<='2020-04-07' and due_date - effective_date+1 = approved_period
    )t1
left join dw_gocash_go_cash_loan_gocash_core_loan t2 on t1.customer_id = t2.customer_id
where t1.apply_time :: timestamp > t2.apply_time :: timestamp
group by loan_id

'''

In [8]:
df2 = data_con.database_gn(query2)
save_data_to_pickle(df2,file_path,'history1.pkl')
df2.shape
df2.head()

Successful connection


(41107, 95)

,loan_id,cnt_loan30,cnt_loan60,cnt_loan90,cnt_loan180,cnt_loan360,cnt_loan,firstapply_curds,lastapply_curds,firstloan_curds,last_loan_curds,max_loan_period,max_loan_period30d,max_loan_period60d,max_loan_period90d,max_loan_period180d,max_loan_period360d,min_loan_period,min_loan_period30d,min_loan_period60d,min_loan_period90d,min_loan_period180d,min_loan_period360d,avg_loan_period,avg_loan_period30d,avg_loan_period60d,avg_loan_period90d,avg_loan_period180d,avg_loan_period360d,cnt_loan_period_is15,cnt_loan_period_is15_30d,cnt_loan_period_is15_60d,cnt_loan_period_is15_90d,cnt_loan_period_is15_180d,cnt_loan_period_is15_360d,cnt_loan_period_is8,cnt_loan_period_is8_30d,cnt_loan_period_is8_60d,cnt_loan_period_is8_90d,cnt_loan_period_is8_180d,cnt_loan_period_is8_360d,rate_loan_period_is15,rate_loan_period_is15_30d,rate_loan_period_is15_60d,rate_loan_period_is15_90d,rate_loan_period_is15_180d,rate_loan_period_is15_360d,rate_loan_period_is8,rate_loan_period_is8_30d,rate_loan_period_is8_60d,rate_loan_period_is8_90d,rate_loan_period_is8_180d,rate_loan_period_is8_360d,cnt_extend_times,cnt_extend_times_30d,cnt_extend_times_60d,cnt_extend_times_90d,cnt_extend_times_180d,cnt_extend_times_360d,max_extend_times,max_extend_times_30d,max_extend_times_60d,max_extend_times_90d,max_extend_times_180d,max_extend_times_360d,sum_extend_times,sum_extend_times_30d,sum_extend_times_60d,sum_extend_times_90d,sum_extend_times_180d,sum_extend_times_360d,avg_extend_times,avg_extend_times_30d,avg_extend_times_60d,avg_extend_times_90d,avg_extend_times_180d,avg_extend_times_360d,cnt_advance_paidoff,cnt_advance_paidoff30d,cnt_advance_paidoff60d,cnt_advance_paidoff90d,cnt_advance_paidoff180d,cnt_advance_paidoff360d,cnt_collection_paidoff,cnt_collection_paidoff30d,cnt_collection_paidoff60d,cnt_collection_paidoff90d,cnt_collection_paidoff180d,cnt_collection_paidoff360d,cnt_paidoff,cnt_paidoff30d,cnt_paidoff60d,cnt_paidoff90d,cnt_paidoff180d,cnt_paidoff360d
0,437422113627406336,1,1,1,1,1,1,22,22,22.0,22.0,22,22.0,22.0,22.0,22.0,22.0,22,22.0,22.0,22.0,22.0,22.0,22.0000000000000000,22.0000000000000000,22.0000000000000000,22.0000000000000000,22.0000000000000000,22.0000000000000000,0,0,0,0,0,0,0,0,0,0,0,0,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1
1,437483994752647168,1,1,1,1,1,1,10,10,10.0,10.0,22,22.0,22.0,22.0,22.0,22.0,22,22.0,22.0,22.0,22.0,22.0,22.0000000000000000,22.0000000000000000,22.0000000000000000,22.0000000000000000,22.0000000000000000,22.0000000000000000,0,0,0,0,0,0,0,0,0,0,0,0,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,437493937882955776,1,1,1,1,1,1,13,13,13.0,13.0,15,15.0,15.0,15.0,15.0,15.0,15,15.0,15.0,15.0,15.0,15.0,15.0000000000000000,15.0000000000000000,15.0000000000000000,15.0000000000000000,15.0000000000000000,15.0000000000000000,1,1,1,1,1,1,0,0,0,0,0,0,1.00000000000000000000,1.00000000000000000000,1.00000000000000000000,1.00000000000000000000,1.00000000000000000000,1.00000000000000000000,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
3,437500670097588224,1,1,1,1,1,1,23,23,22.0,22.0,22,22.0,22.0,22.0,22.0,22.0,22,22.0,22.0,22.0,22.0,22.0,22.0000000000000000,22.0000000000000000,22.0000000000000000,22.0000000000000000,22.0000000000000000,22.0000000000000000,0,0,0,0,0,0,0,0,0,0,0,0,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0E-20,0E-20,0E-20,0E-20,0E-20,0E-20,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0
4,437505288470700032,1,2,2,2,2,2,32,3,32.0,3.0,29,29.0,29.0,29.0,29.0,29.0,15,29.0,15.0,15.0,15.0,15.0,22.0000000000000000,29.0000000000000000,22.00000000

In [3]:
query3 = '''
SELECT t1.loan_id
--最大逾期天数
, max(late_days) as max_his_latedays
, max(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 then late_days else null end) as max_his_latedays_30d 
, max(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 then late_days else null end) as max_his_latedays_60d 
, max(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 then late_days else null end) as max_his_latedays_90d 
, max(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then late_days else null end) as max_his_latedays_180d 
, max(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then late_days else null end) as max_his_latedays_360d 
--最小逾期天数
, min(late_days) as min_his_latedays
, min(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 	then late_days else null end) as min_his_latedays_30d 
, min(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 	then late_days else null end) as min_his_latedays_60d 
, min(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 	then late_days else null end) as min_his_latedays_90d 
, min(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then late_days else null end) as min_his_latedays_180d 
, min(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then late_days else null end) as min_his_latedays_360d 
----总共
, sum(late_days) as sum_his_latedays
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 then late_days else null end) as sum_his_latedays_30d 
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 then late_days else null end) as sum_his_latedays_60d 
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 then late_days else null end) as sum_his_latedays_90d 
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then late_days else null end) as sum_his_latedays_180d 
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then late_days else null end) as sum_his_latedays_360d 
----平均
, sum(late_days)/count(distinct t2.id) as avg_his_latedays
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30  then late_days else null end)/
    count(distinct case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 then t2.id else null end)as avg_his_latedays_30d 
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 then late_days else null end)/
	count(distinct case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 then t2.id else null end)as avg_his_latedays_60d 
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 then late_days else null end)/
	count(distinct case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 then t2.id else null end)as avg_his_latedays_90d 
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then late_days else null end)/
	count(distinct case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then t2.id else null end)as avg_his_latedays_180d 
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then late_days else null end)/
	count(distinct case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then t2.id else null end)as avg_his_latedays_360d 
from (SELECT id as loan_id, apply_time, effective_date, customer_id
      FROM dw_gocash_go_cash_loan_gocash_core_loan
      where return_flag = 'true' and due_date::date>='2020-03-20' and due_date::date<='2020-04-07' and due_date - effective_date+1 = approved_period
    )t1
LEFT JOIN (SELECT id, apply_time, customer_id FROM dw_gocash_go_cash_loan_gocash_core_loan) t2 on t1.customer_id = t2.customer_id
LEFT JOIN (SELECT * FROM public.dw_gocash_go_cash_loan_gocash_core_loan_pay_flow WHERE status = 'SUCCESS' ) t3 ON t2.id = t3.loan_id 
WHERE t1.apply_time :: timestamp > t2.apply_time :: timestamp
group by 1
'''

In [4]:
df3 = data_con.database_gn(query3)
save_data_to_pickle(df3,file_path,'history2.pkl')
df3.shape
df3.head()

Successful connection


(41107, 25)

,loan_id,max_his_latedays,max_his_latedays_30d,max_his_latedays_60d,max_his_latedays_90d,max_his_latedays_180d,max_his_latedays_360d,min_his_latedays,min_his_latedays_30d,min_his_latedays_60d,min_his_latedays_90d,min_his_latedays_180d,min_his_latedays_360d,sum_his_latedays,sum_his_latedays_30d,sum_his_latedays_60d,sum_his_latedays_90d,sum_his_latedays_180d,sum_his_latedays_360d,avg_his_latedays,avg_his_latedays_30d,avg_his_latedays_60d,avg_his_latedays_90d,avg_his_latedays_180d,avg_his_latedays_360d
0,437422113627406336,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
1,437483994752647168,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
2,437493937882955776,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
3,437500670097588224,1,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,1.0,1.0,1.0
4,437505288470700032,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [5]:
query4 = '''
SELECT t1.loan_id
, sum(case when date_part('hour', t3.paid_off_time) between 0 and 5 then 1 else 0 end) as cnt_paidoffhour_0to5
, sum(case when date_part('hour', t3.paid_off_time) between 6 and 12 then 1 else 0 end) as cnt_paidoffhour_6to12
, sum(case when date_part('hour', t3.paid_off_time) between 13 and 18 then 1 else 0 end) as cnt_paidoffhour_13to18
, sum(case when date_part('hour', t3.paid_off_time) between 19 and 23 then 1 else 0 end) as cnt_paidoffhour_19to24
, 1.0 * sum(case when date_part('hour', t3.paid_off_time) between 0 and 5 then 1 else 0 end)/count(t3.paid_off_time) as rate_paidoffhour_0to5
, 1.0 * sum(case when date_part('hour', t3.paid_off_time) between 6 and 12 then 1 else 0 end)/count(t3.paid_off_time) as rate_paidoffhour_6to12
, 1.0 * sum(case when date_part('hour', t3.paid_off_time) between 13 and 18 then 1 else 0 end)/count(t3.paid_off_time) as rate_paidoffhour_13to18
, 1.0 * sum(case when date_part('hour', t3.paid_off_time) between 19 and 23 then 1 else 0 end)/count(t3.paid_off_time) as rate_paidoffhour_19to24
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and date_part('hour', t3.paid_off_time) between 0 and 5 then 1 else 0 end) as cnt_paidoffhour_0to5_30d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and date_part('hour', t3.paid_off_time) between 6 and 12 then 1 else 0 end) as cnt_paidoffhour_6to12_30d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and date_part('hour', t3.paid_off_time) between 13 and 18 then 1 else 0 end) as cnt_paidoffhour_13to18_30d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and date_part('hour', t3.paid_off_time) between 19 and 23 then 1 else 0 end) as cnt_paidoffhour_19to23_30d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and date_part('hour', t3.paid_off_time) between 0 and 5 then 1 else 0 end) as cnt_paidoffhour_0to5_60d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and date_part('hour', t3.paid_off_time) between 6 and 12 then 1 else 0 end) as cnt_paidoffhour_6to12_60d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and date_part('hour', t3.paid_off_time) between 13 and 18 then 1 else 0 end) as cnt_paidoffhour_13to18_60d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and date_part('hour', t3.paid_off_time) between 19 and 23 then 1 else 0 end) as cnt_paidoffhour_19to23_60d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and date_part('hour', t3.paid_off_time) between 0 and 5 then 1 else 0 end) as cnt_paidoffhour_0to5_90d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and date_part('hour', t3.paid_off_time) between 6 and 12 then 1 else 0 end) as cnt_paidoffhour_6to12_90d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and date_part('hour', t3.paid_off_time) between 13 and 18 then 1 else 0 end) as cnt_paidoffhour_13to18_90d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and date_part('hour', t3.paid_off_time) between 19 and 23 then 1 else 0 end) as cnt_paidoffhour_19to23_90d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and date_part('hour', t3.paid_off_time) between 0 and 5 then 1 else 0 end) as cnt_paidoffhour_0to5_180d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and date_part('hour', t3.paid_off_time) between 6 and 12 then 1 else 0 end) as cnt_paidoffhour_6to12_180d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and date_part('hour', t3.paid_off_time) between 13 and 18 then 1 else 0 end) as cnt_paidoffhour_13to18_180d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and date_part('hour', t3.paid_off_time) between 19 and 23 then 1 else 0 end) as cnt_paidoffhour_19to23_180d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and date_part('hour', t3.paid_off_time) between 0 and 5 then 1 else 0 end) as cnt_paidoffhour_0to5_360d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and date_part('hour', t3.paid_off_time) between 6 and 12 then 1 else 0 end) as cnt_paidoffhour_6to12_360d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and date_part('hour', t3.paid_off_time) between 13 and 18 then 1 else 0 end) as cnt_paidoffhour_13to18_360d
, sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and date_part('hour', t3.paid_off_time) between 19 and 23 then 1 else 0 end) as cnt_paidoffhour_19to23_360d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 then t3.paid_off_time else null end) > 0 
	    then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and date_part('hour', t3.paid_off_time) between 0 and 5 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 then t3.paid_off_time else null end)
        else null end as rate_paidoffhour_0to5_30d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 then t3.paid_off_time else null end) > 0 
        then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and date_part('hour', t3.paid_off_time) between 6 and 12 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 then t3.paid_off_time else null end ) 
        else null end as rate_paidoffhour_6to12_30d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 then t3.paid_off_time else null end) > 0 
        then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and date_part('hour', t3.paid_off_time) between 13 and 18 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 then t3.paid_off_time else null end)
        else null end as rate_paidoffhour_13to18_30d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 then t3.paid_off_time else null end) > 0 
        then  1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 and date_part('hour', t3.paid_off_time) between 19 and 23 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 30 then t3.paid_off_time else null end) 
        else null end as rate_paidoffhour_19to23_30d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 then t3.paid_off_time else null end) > 0 
	    then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and date_part('hour', t3.paid_off_time) between 0 and 5 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 then t3.paid_off_time else null end)
        else null end as rate_paidoffhour_0to5_60d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 then t3.paid_off_time else null end) > 0 
        then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and date_part('hour', t3.paid_off_time) between 6 and 12 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 then t3.paid_off_time else null end ) 
        else null end as rate_paidoffhour_6to12_60d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 then t3.paid_off_time else null end) > 0 
        then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and date_part('hour', t3.paid_off_time) between 13 and 18 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 then t3.paid_off_time else null end)
        else null end as rate_paidoffhour_13to18_60d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 then t3.paid_off_time else null end) > 0 
        then  1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 and date_part('hour', t3.paid_off_time) between 19 and 23 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 60 then t3.paid_off_time else null end) 
        else null end as rate_paidoffhour_19to23_60d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 then t3.paid_off_time else null end) > 0 
	    then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and date_part('hour', t3.paid_off_time) between 0 and 5 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 then t3.paid_off_time else null end)
        else null end as rate_paidoffhour_0to5_90d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 then t3.paid_off_time else null end) > 0 
        then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and date_part('hour', t3.paid_off_time) between 6 and 12 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 then t3.paid_off_time else null end ) 
        else null end as rate_paidoffhour_6to12_90d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 then t3.paid_off_time else null end) > 0 
        then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and date_part('hour', t3.paid_off_time) between 13 and 18 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 then t3.paid_off_time else null end)
        else null end as rate_paidoffhour_13to18_90d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 then t3.paid_off_time else null end) > 0 
        then  1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 and date_part('hour', t3.paid_off_time) between 19 and 23 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 90 then t3.paid_off_time else null end) 
        else null end as rate_paidoffhour_19to23_90d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then t3.paid_off_time else null end) > 0 
	    then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and date_part('hour', t3.paid_off_time) between 0 and 5 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then t3.paid_off_time else null end)
        else null end as rate_paidoffhour_0to5_180d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then t3.paid_off_time else null end) > 0 
        then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and date_part('hour', t3.paid_off_time) between 6 and 12 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then t3.paid_off_time else null end ) 
        else null end as rate_paidoffhour_6to12_180d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then t3.paid_off_time else null end) > 0 
        then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and date_part('hour', t3.paid_off_time) between 13 and 18 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then t3.paid_off_time else null end)
        else null end as rate_paidoffhour_13to18_180d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then t3.paid_off_time else null end) > 0 
        then  1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 and date_part('hour', t3.paid_off_time) between 19 and 23 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 180 then t3.paid_off_time else null end) 
        else null end as rate_paidoffhour_19to23_180d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then t3.paid_off_time else null end) > 0 
	    then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and date_part('hour', t3.paid_off_time) between 0 and 5 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then t3.paid_off_time else null end)
        else null end as rate_paidoffhour_0to5_360d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then t3.paid_off_time else null end) > 0 
        then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and date_part('hour', t3.paid_off_time) between 6 and 12 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then t3.paid_off_time else null end ) 
        else null end as rate_paidoffhour_6to12_360d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then t3.paid_off_time else null end) > 0 
        then 1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and date_part('hour', t3.paid_off_time) between 13 and 18 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then t3.paid_off_time else null end)
        else null end as rate_paidoffhour_13to18_360d
, case when count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then t3.paid_off_time else null end) > 0 
        then  1.0 * sum(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 and date_part('hour', t3.paid_off_time) between 19 and 23 then 1 else 0 end)/
        count(case when (t1.apply_time :: date - t2.apply_time :: date) <= 360 then t3.paid_off_time else null end) 
        else null end as rate_paidoffhour_19to23_360d
FROM (SELECT id as loan_id, apply_time, effective_date, customer_id
      FROM dw_gocash_go_cash_loan_gocash_core_loan
      where return_flag = 'true' and due_date::date>='2020-03-20' and due_date::date<='2020-04-07' and due_date - effective_date+1 = approved_period)t1 -- and customer_id = 352418834367426560
LEFT JOIN (SELECT id, apply_time, customer_id FROM dw_gocash_go_cash_loan_gocash_core_loan ) t2 on t1.customer_id = t2.customer_id
LEFT JOIN (SELECT * FROM public.dw_gocash_go_cash_loan_gocash_core_loan_pay_flow WHERE status = 'SUCCESS' ) t3 ON t2.id = t3.loan_id 
WHERE t1.apply_time :: timestamp > t2.apply_time :: timestamp
group by 1
'''

In [6]:
df4 = data_con.database_gn(query4)
save_data_to_pickle(df4,file_path,'history3.pkl')
df4.shape
df4.head()

Successful connection


(41107, 49)

,loan_id,cnt_paidoffhour_0to5,cnt_paidoffhour_6to12,cnt_paidoffhour_13to18,cnt_paidoffhour_19to24,rate_paidoffhour_0to5,rate_paidoffhour_6to12,rate_paidoffhour_13to18,rate_paidoffhour_19to24,cnt_paidoffhour_0to5_30d,cnt_paidoffhour_6to12_30d,cnt_paidoffhour_13to18_30d,cnt_paidoffhour_19to23_30d,cnt_paidoffhour_0to5_60d,cnt_paidoffhour_6to12_60d,cnt_paidoffhour_13to18_60d,cnt_paidoffhour_19to23_60d,cnt_paidoffhour_0to5_90d,cnt_paidoffhour_6to12_90d,cnt_paidoffhour_13to18_90d,cnt_paidoffhour_19to23_90d,cnt_paidoffhour_0to5_180d,cnt_paidoffhour_6to12_180d,cnt_paidoffhour_13to18_180d,cnt_paidoffhour_19to23_180d,cnt_paidoffhour_0to5_360d,cnt_paidoffhour_6to12_360d,cnt_paidoffhour_13to18_360d,cnt_paidoffhour_19to23_360d,rate_paidoffhour_0to5_30d,rate_paidoffhour_6to12_30d,rate_paidoffhour_13to18_30d,rate_paidoffhour_19to23_30d,rate_paidoffhour_0to5_60d,rate_paidoffhour_6to12_60d,rate_paidoffhour_13to18_60d,rate_paidoffhour_19to23_60d,rate_paidoffhour_0to5_90d,rate_paidoffhour_6to12_90d,rate_paidoffhour_13to18_90d,rate_paidoffhour_19to23_90d,rate_paidoffhour_0to5_180d,rate_paidoffhour_6to12_180d,rate_paidoffhour_13to18_180d,rate_paidoffhour_19to23_180d,rate_paidoffhour_0to5_360d,rate_paidoffhour_6to12_360d,rate_paidoffhour_13to18_360d,rate_paidoffhour_19to23_360d
0,437422113627406336,1,0,0,0,1.00000000000000000000,0E-20,0E-20,0E-20,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20
1,437483994752647168,1,0,0,0,1.00000000000000000000,0E-20,0E-20,0E-20,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20
2,437493937882955776,0,0,1,0,0E-20,0E-20,1.00000000000000000000,0E-20,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20
3,437500670097588224,1,0,0,0,1.00000000000000000000,0E-20,0E-20,0E-20,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20,1.00000000000000000000,0E-20,0E-20,0E-20
4,437505288470700032,2,0,1,1,0.50000000000000000000,0E-20,0.25000000000000000000,0.25000000000000000000,1,0,0,0,2,0,1,1,2,0,1,1,2,0,1,1,2,0,1,1,1.00000000000000000000,0E-20,0E-20,0E-20,0.50000000000000000000,0E-20,0.25000000000000000000,0.25000000000000000000,0.50000000000000000000,0E-20,0.25000000000000000000,0.25000000000000000000,0.50000000000000000000,0E-20,0.25000000000000000000,0.25000000000000000000,0.50000000000000000000,0E-20,0.25000000000000000000,0.25000000000000000000


# izi

In [3]:
query5 = '''
select *
from (
select t1.loan_id,
concat(year, month, day) as apply_time, 
date(t1.apply_time) - interval '1 day' as pre_dt,
t3.message,
row_number()over(partition by t1.loan_id order by t3.create_time desc) as rank1
from (SELECT id as loan_id, apply_time, effective_date, customer_id,
     to_char(apply_time, 'YYYY') as year , to_char(apply_time, 'MM') as month, to_char(apply_time, 'DD') as day
      FROM dw_gocash_go_cash_loan_gocash_core_loan
      where return_flag = 'true' and due_date::date>='2020-03-20' and due_date::date<='2020-04-07' and due_date - effective_date+1 = approved_period)t1 
left join 
(select distinct orderno,customerid,createtime
from rt_risk_mongo_gocash_riskreport) t2 
on t1.loan_id::varchar = t2.orderno
left join 
(select * from gocash_oss_inquiries_v4
where status = 'OK') t3 ON t2.customerid = t3.customer_id
WHERE date(t2.createtime)>=date(t3.create_time) or t3.create_time is null) t 
where rank1 = 1;
'''

In [4]:
df5 = data_con.database_gn(query5)
save_data_to_pickle(df5,file_path,'izi_rawdata.pkl')
df5.shape
df5.head()
df5['message'].isna().sum()
# 40750   10987

Successful connection


(40639, 5)

,loan_id,apply_time,pre_dt,message,rank1
0,437422113627406336,20200221,2020-02-20,None,1
1,437483994752647168,20200221,2020-02-20,None,1
2,437493937882955776,20200221,2020-02-20,None,1
3,437500670097588224,20200221,2020-02-20,None,1
4,437505288470700032,20200221,2020-02-20,None,1


10979

In [7]:
# 从json/dict形式提取出所有日期
def extract_date(dict_ori):
    s=''
    for i in dict_ori.values():
        s = s+","+','.join(i)
    string = s.lstrip(',')
    lst = string.split(',')
    return lst

import datetime

def date_to_daysdiff(row, length, col):
    if row[col] == ['']:
        date_diff=[]
    elif row[col] == []:
        date_diff=[]
    else:
        date_diff=[]
        for i in range(length):
            start = row[col][i]
            end = row['apply_time']
            d1 = datetime.datetime.strptime(start, '%Y%m%d')
            d2 = datetime.datetime.strptime(str(end), '%Y%m%d')
            diff = (d2-d1).days
            #print(diff)
            date_diff.append(str(diff))
            #print(date_diff)
            #date_d = date_diff.astype(str)
    return date_diff

In [8]:
izi_df = df5[~df5['message'].isna()]
izi_df.shape
try1= dp.from_json(izi_df, 'message')
try2= dp.from_json(try1, 'detail')
try3 = try2.copy()
try3['AA'] = try3['A'].apply(lambda x: extract_date(x))
try3['BB'] = try3['B'].apply(lambda x: extract_date(x))
try3['CC'] = try3['C'].apply(lambda x: extract_date(x))
# try3.head()

try4 = try3.copy()
try4.pre_dt = try4.pre_dt.astype(str)
for index, row in try4.iterrows():
    try:
        try4.loc[index, 'AA'] = str(try4.loc[index,'AA']).replace('Within24hours',try4.loc[index, 'pre_dt'].replace('-',''))
        try4.loc[index, 'BB'] = str(try4.loc[index,'BB']).replace('Within24hours',try4.loc[index, 'pre_dt'].replace('-',''))
        try4.loc[index, 'CC'] = str(try4.loc[index,'CC']).replace('Within24hours',try4.loc[index, 'pre_dt'].replace('-',''))
    except:
        print(index)
        pass

import ast
try4['AA'] = try4['AA'].apply(lambda x:ast.literal_eval(x))
try4['BB'] = try4['BB'].apply(lambda x:ast.literal_eval(x))
try4['CC'] = try4['CC'].apply(lambda x:ast.literal_eval(x))


for row in range(try4.shape[0]):
    a=try4.iloc[row]
    try4.loc[row,'A_daysdiff']= ','.join(date_to_daysdiff(a, len(a['AA']), 'AA'))
    try4.loc[row,'B_daysdiff']= ','.join(date_to_daysdiff(a, len(a['BB']), 'BB'))
    try4.loc[row,'C_daysdiff']= ','.join(date_to_daysdiff(a, len(a['CC']), 'CC'))


try4.A_daysdiff = try4.A_daysdiff.replace('', '-9999')
try4.B_daysdiff = try4.B_daysdiff.replace('', '-9999')
try4.C_daysdiff = try4.C_daysdiff.replace('', '-9999')
try5 = try4.copy()
try5['A_3d'] = try5['A_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=3 and int(a)>=0]))
try5['A_7d'] = try5['A_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=7 and int(a)>=0]))
try5['A_14d'] = try5['A_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=14 and int(a)>=0]))
try5['A_21d'] = try5['A_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=21 and int(a)>=0]))
try5['A_30d'] = try5['A_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=30 and int(a)>=0]))
try5['A_60d'] = try5['A_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=60 and int(a)>=0]))
try5['A_90d'] = try5['A_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=90 and int(a)>=0]))
try5['A_180d'] = try5['A_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=180 and int(a)>=0]))
try5['A_360d'] = try5['A_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=360 and int(a)>=0]))

try5['B_3d'] = try5['B_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=3 and int(a)>=0]))
try5['B_7d'] = try5['B_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=7 and int(a)>=0]))
try5['B_14d'] = try5['B_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=14 and int(a)>=0]))
try5['B_21d'] = try5['B_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=21 and int(a)>=0]))
try5['B_30d'] = try5['B_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=30 and int(a)>=0]))
try5['B_60d'] = try5['B_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=60 and int(a)>=0]))
try5['B_90d'] = try5['B_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=90 and int(a)>=0]))
try5['B_180d'] = try5['B_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=180 and int(a)>=0]))
try5['B_360d'] = try5['B_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=360 and int(a)>=0]))

try5['C_3d'] = try5['C_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=3 and int(a)>=0]))
try5['C_7d'] = try5['C_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=7 and int(a)>=0]))
try5['C_14d'] = try5['C_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=14 and int(a)>=0]))
try5['C_21d'] = try5['C_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=21 and int(a)>=0]))
try5['C_30d'] = try5['C_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=30 and int(a)>=0]))
try5['C_60d'] = try5['C_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=60 and int(a)>=0]))
try5['C_90d'] = try5['C_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=90 and int(a)>=0]))
try5['C_180d'] = try5['C_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=180 and int(a)>=0]))
try5['C_360d'] = try5['C_daysdiff'].apply(lambda x : len([a for a in x.split(',') if int(a)<=360 and int(a)>=0]))
try5.head()
drop_list = ['apply_time', 'pre_dt', 'total', 'A', 'C', 'B', 'AA', 'BB',
       'CC', 'A_daysdiff', 'B_daysdiff', 'C_daysdiff']
try5.drop(drop_list,axis = 1,inplace= True)
save_data_to_pickle(try5,file_path,'izi.pkl')
try5.head()

(29660, 5)

new columns done
detail
total
run time = 0.08648180961608887
new columns done
C
A
B
run time = 0.09502911567687988


,loan_id,apply_time,pre_dt,rank1,total,C,A,B,AA,BB,CC,A_daysdiff,B_daysdiff,C_daysdiff,A_3d,A_7d,A_14d,A_21d,A_30d,A_60d,A_90d,A_180d,A_360d,B_3d,B_7d,B_14d,B_21d,B_30d,B_60d,B_90d,B_180d,B_360d,C_3d,C_7d,C_14d,C_21d,C_30d,C_60d,C_90d,C_180d,C_360d
0,439123349753724928,20200225,2020-02-24,1,15,{'a22dfe': ['20191202']},"{'5b7734': ['20191124'], '114795': ['20191124'...","{'f2ead5': ['20191031', '20191124'], '44000b':...","[20191124, 20191124, 20191031, 20191119, 20191...","[20191031, 20191124, 20200224, 20200224, 20191...",[20191202],"93,93,117,98,60,59,43,26","117,93,1,1,93,89",85,0,0,0,0,1,4,4,8,8,2,2,2,2,2,2,3,6,6,0,0,0,0,0,0,1,1,1
1,439821174947020800,20200227,2020-02-26,1,21,{},"{'278027': ['20191027', '20191028'], '5b7734':...","{'e27967': ['20200219'], '7c78b0': ['20200219']}","[20191027, 20191028, 20190608, 20190908, 20191...","[20200219, 20200219]",[],"123,122,264,172,123,22,11,182,147,107,77,30,10...","8,8",-9999,1,3,4,4,7,7,8,17,19,0,0,2,2,2,2,2,2,2,0,0,0,0,0,0,0,0,0
2,439833037306953728,20200227,2020-02-26,1,53,{},"{'278027': ['20190716', '20190717'], '392aec':...","{'b255f6': ['20191121', '20200122'], 'f2ead5':...","[20190716, 20190717, 20190829, 20190330, 20191...","[20191121, 20200122, 20190826, 20190923, 20191...",[],"226,225,182,334,135,297,172,153,137,121,104,89...","98,36,185,157,127,94,151,224,219",-9999,1,2,2,3,4,7,12,23,44,0,0,0,0,0,1,1,6,9,0,0,0,0,0,0,0,0,0
3,439833517156114432,20200227,2020-02-26,1,24,{},"{'278027': ['20191030'], '5b7734': ['20190507'...","{'b255f6': ['20200131', '20200218'], 'a368de':...","[20191030, 20190507, 20190926, 20190317, 20190...","[20200131, 20200218, 20191113]",[],"120,296,154,347,298,348,287,188,298,106,151,29...","27,9,106",-9999,0,0,1,1,1,1,2,10,21,0,0,1,1,2,2,2,3,3,0,0,0,0,0,0,0,0,0
4,439837312963485696,20200227,2020-02-26,1,15,"{'3baaf6': ['20191127'], 'e8348a': ['20191121']}","{'5b7734': ['20191115'], 'cc5780': ['20191030'...","{'1109f1': ['20191018'], 'f2ead5': ['20190912'...","[20191115, 20191030, 20191118, 20190910, 20191...","[20191018, 20190912, 20200220, 20191105]","[20191127, 20191121]","104,120,101,170,69,50,29,87,31","132,168,7,114","92,98",0,0,0,0,1,3,5,9,9,0,1,1,1,1,1,1,4,4,0,0,0,0,0,0,0,2,2


,loan_id,rank1,A_3d,A_7d,A_14d,A_21d,A_30d,A_60d,A_90d,A_180d,A_360d,B_3d,B_7d,B_14d,B_21d,B_30d,B_60d,B_90d,B_180d,B_360d,C_3d,C_7d,C_14d,C_21d,C_30d,C_60d,C_90d,C_180d,C_360d
0,439123349753724928,1,0,0,0,0,1,4,4,8,8,2,2,2,2,2,2,3,6,6,0,0,0,0,0,0,1,1,1
1,439821174947020800,1,1,3,4,4,7,7,8,17,19,0,0,2,2,2,2,2,2,2,0,0,0,0,0,0,0,0,0
2,439833037306953728,1,1,2,2,3,4,7,12,23,44,0,0,0,0,0,1,1,6,9,0,0,0,0,0,0,0,0,0
3,439833517156114432,1,0,0,1,1,1,1,2,10,21,0,0,1,1,2,2,2,3,3,0,0,0,0,0,0,0,0,0
4,439837312963485696,1,0,0,0,0,1,3,5,9,9,0,1,1,1,1,1,1,4,4,0,0,0,0,0,0,0,2,2


# 线上模型分

In [1]:
query6 = '''
select t1.loan_id,
t2.ruleresult as oldUserModelV2
from (SELECT id as loan_id, apply_time, effective_date, customer_id,
     to_char(apply_time, 'YYYY') as year , to_char(apply_time, 'MM') as month, to_char(apply_time, 'DD') as day
      FROM dw_gocash_go_cash_loan_gocash_core_loan
      where return_flag = 'true' and due_date::date>='2020-03-20' and due_date::date<='2020-04-07' and due_date - effective_date+1 = approved_period)t1 
left join 
(select * from rt_risk_mongo_gocash_riskreport
where ruleresultname = 'oldUserModelV2Score') t2 
on t1.loan_id::varchar = t2.orderno
'''

In [6]:
df6 = data_con.database_gn(query6)
save_data_to_pickle(df6,file_path,'v2_score.pkl')
df6.shape
df6.head()
# df6.info()

Successful connection


(40855, 2)

,loan_id,oldusermodelv2
0,437618487241121792,None
1,437654581873836032,None
2,437872384908296192,None
3,438009474178260992,None
4,438369463774380032,None


# APP

In [6]:
query7 = '''
select loan_id,packages_cleaned_2
from 
(select t1.loan_id,t2.packages_cleaned_2,
row_number()over(partition by t1.loan_id order by t2.create_time desc) as rank1
from 
(SELECT id as loan_id ,customer_id FROM dw_gocash_go_cash_loan_gocash_core_loan
  where return_flag = 'true' and due_date::date>='2020-03-20' and due_date::date<='2020-04-07' and due_date - effective_date+1 = approved_period)t1 
left join gocash_loan_risk_program_packages t2 
on t1.customer_id::varchar = t2.customer_id) t 
where rank1 = 1
'''

In [7]:
df7 = data_con.database_gn(query7)
save_data_to_pickle(df7,file_path,'APP.pkl')
df7.shape
df7.head()

Successful connection


(40765, 2)

,loan_id,packages_cleaned_2
0,437422113627406336,"[{""appName"":""RekamLayar"",""installTime"":""123073..."
1,437483994752647168,"[{""appName"":""WidgetGalaxyEssentials"",""installT..."
2,437493937882955776,"[{""appName"":""YouTube"",""installTime"":""123073920..."
3,437500670097588224,"[{""appName"":""Widget Galaxy Essentials"",""instal..."
4,437505288470700032,"[{""appName"":""WidgetGalaxyEssentials"",""installT..."


In [22]:
time1 = time.time()
df7= load_data_from_pickle(file_path, 'APP.pkl')
df71 = df7[~df7['packages_cleaned_2'].isna()]
df71 = df71[df71['packages_cleaned_2'] != '']
df_app = dp.from_json(df71, 'packages_cleaned_2')
df_app = df_app[['loan_id','appName','packageName']]
df_app.head()
save_data_to_pickle(df_app,file_path,'df_app.pkl')
time2 = time.time() 
print('run_time: ' ,time2-time1)

new columns done
versionCode
appName
installTime
versionName
packageName
temp
run time = 15.994520902633667


,loan_id,appName,packageName
0,437422113627406336,RekamLayar,com.miui.screenrecorder
1,437422113627406336,AdaKami,com.idnada.loan
2,437422113627406336,YouTube,com.google.android.youtube
3,437422113627406336,KlikKami,com.ayo.klikkami
4,437422113627406336,Easycash,com.fintopia.idnEasycash.google


run_time:  28.309434175491333


拆分数据集

In [6]:
df_app = load_data_from_pickle(file_path,'df_app.pkl')
df_app.shape
y = load_data_from_pickle(file_path,'y.pkl')
y['sample_flag'] = y['due_date'].apply(lambda x: 0 if x<pd.to_datetime('2020-04-01') else 1)
y_train = y[y['sample_flag']==0][['loan_id']]
df_app_train1 = y_train.merge(df_app,on='loan_id',how = 'left')
save_data_to_pickle(df_app_train1,file_path,'df_app_train1.pkl')
df_app_train1.shape

(3768952, 2)

(2658542, 2)

 1）生成频数字典

In [8]:
time1 = time.time()
high_cut = 0.4 
mid_cut = 0.2
tfidf_cut = 0.1
app_data1 = load_data_from_pickle(file_path,'df_app_train1.pkl')
app_freq = pd.DataFrame(app_data1['appName'].value_counts()).reset_index()
print('训练集的不同app数量: ' , app_freq.shape[0])
app_freq.rename(columns={'index': 'app', 'appName': 'freq'}, inplace=True)
cnt_loan = app_data1['loan_id'].nunique()
app_freq['rate_freq'] = app_freq['freq'].apply(lambda x: x/cnt_loan)
app_tfidf = app_freq[app_freq['rate_freq']>=float('%f'%tfidf_cut)]
print('可用于tf_idf的app总数是(安装率超过 %f)： '%tfidf_cut, app_tfidf.shape[0])
app_tfidf.to_csv(file_path + 'app_for_tfidf.csv', index = False)
app_freq['high_freq_app'] = app_freq['rate_freq'].apply(lambda x: 1 if x >= float('%f'%high_cut) else 0)
print('高频APP个数', app_freq['high_freq_app'].sum())
app_freq['mid_freq_app'] = app_freq['rate_freq'].apply(lambda x: 1 if (float('%f'%mid_cut)<= x <float('%f'%high_cut)) else 0)
print('中频APP个数', app_freq['mid_freq_app'].sum())
app_freq['low_freq_app'] = app_freq['freq'].apply(lambda x: 1 if x <= 1 else 0)
print('低频APP个数(仅一人安装)', app_freq['low_freq_app'].sum())
app_freq.to_csv(file_path  + 'app_freq_train_dict.csv', index=False)
time2 = time.time() 
print('run_time: ' ,time2-time1)
time.sleep(0.1)

训练集的不同app数量:  52860
可用于tf_idf的app总数是(安装率超过 0.100000)：  188
高频APP个数 36
中频APP个数 56
低频APP个数(仅一人安装) 25338
run_time:  153.74339890480042


2) 生成频数相关的变量

In [10]:
time1 = time.time()
time.sleep(0.1)
df_app = load_data_from_pickle(file_path,'df_app.pkl')
count_app = Counter(df_app['loan_id'])
app_freq = pd.read_csv(file_path  + 'app_freq_train_dict.csv')
train1 = pd.merge(df_app, app_freq, left_on='appName', right_on='app', how = 'left')
var_app = train1.groupby(['loan_id'])['high_freq_app', 'mid_freq_app', 'low_freq_app'].agg('sum').reset_index()
var_app['cnt_app'] = var_app['loan_id'].map(lambda x: count_app[x])
list1 = ['high_freq_app', 'mid_freq_app', 'low_freq_app']
for i in list1:
    var_app['rate_' + i] = var_app.apply(lambda x: x[i]/x['cnt_app'], axis=1)
var_app.to_csv(file_path + 'var_app_freq.csv', index=False)
var_dict_app_freq = dp.VarDict(var_app.drop('loan_id', axis = 1), data_sorce='app_freq')
var_dict_app_freq.to_csv(os.path.join(file_path, 'var_dict_app_freq.csv'), index = False)
print('app频数变量: ', var_app.shape)
var_app.head()
time2 = time.time() 
print('run_time: ' ,time2-time1)

/Users/yantingting/opt/anaconda3/envs/modeling/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


app频数变量:  (35959, 8)


,loan_id,high_freq_app,mid_freq_app,low_freq_app,cnt_app,rate_high_freq_app,rate_mid_freq_app,rate_low_freq_app
0,437422113627406336,31.0,20.0,0.0,104,0.298077,0.192308,0.000000
1,437483994752647168,31.0,17.0,0.0,102,0.303922,0.166667,0.000000
2,437493937882955776,29.0,16.0,0.0,79,0.367089,0.202532,0.000000
3,437500670097588224,18.0,32.0,0.0,119,0.151261,0.268908,0.000000
4,437505288470700032,29.0,17.0,2.0,106,0.273585,0.160377,0.018868


run_time:  450.12580013275146


1）生成 idf 字典

In [14]:
time1 = time.time()
time.sleep(0.5)
# df_app_train1 = load_data_from_pickle(file_path,'df_app_train1.pkl')
app_idf_data = df_app_train1[['loan_id','appName']].copy()
app_idf = pd.pivot_table(app_idf_data.drop_duplicates(), index = 'appName', values = 'loan_id', aggfunc = 'count').reset_index()
app_idf.rename(columns={'loan_id': 'cnt_idf'}, inplace=True)
cnt_document = app_idf_data['loan_id'].nunique()
print(1)
app_idf['idf'] = app_idf['cnt_idf'].apply(lambda x: math.log10(cnt_document/(x+1)))
app_idf[['appName', 'idf']].to_csv(file_path + 'app_idf_train1_dict.csv', index=False)
app_idf.shape
app_idf.head()
time2 = time.time() 
print('run_time: ' ,time2-time1)

1


(52860, 3)

,appName,cnt_idf,idf
0,Universal TV Remote,9,3.459061
1,#DRIVE,1,4.158031
2,#DWPX,1,4.158031
3,#Dramania,1,4.158031
4,( ͡° ͜ʖ ͡°) PugWars,1,4.158031


run_time:  268.2483420372009


In [23]:
time1 = time.time()
time.sleep(0.5)
df_app = load_data_from_pickle(file_path,'df_app.pkl')
print(df_app.shape)
dict_tfidf = pd.read_csv(file_path + 'app_idf_train1_dict.csv')
app_tfidf_data = df_app.copy()
# 计算每个loan的app个数
cnt_app = Counter(app_tfidf_data['loan_id'])
# 计算TF
app_tfidf =  pd.pivot_table(app_tfidf_data, index=['loan_id', 'appName'], values='packageName',aggfunc='count' ,fill_value=0).reset_index()
app_tfidf.rename(columns = {'packageName': 'cnt_tf'},  inplace=True)
app_tfidf['cnt_app'] = app_tfidf['loan_id'].map(lambda x: cnt_app[x])
app_tfidf['app_tf'] = app_tfidf.apply(lambda x: x['cnt_tf']/x['cnt_app'], axis = 1)

# 计算IDF
app_tfidf = pd.merge(app_tfidf, dict_tfidf, on='appName', how = 'left')
app_tfidf['tf_idf'] = app_tfidf.apply(lambda x: x['app_tf']*x['idf'], axis=1)

app_for_tfidf = pd.read_csv(file_path + 'app_for_tfidf.csv')
app_tfidf_list = app_for_tfidf['app'].values.tolist()
app_tfidf1=app_tfidf[app_tfidf['appName'].isin(app_tfidf_list)]
var_app_tfidf = pd.pivot_table(app_tfidf1, index='loan_id', columns = 'appName',values='tf_idf', aggfunc='sum', fill_value=0).reset_index()
var_app_tfidf = pd.merge(df_app[['loan_id']].drop_duplicates(),var_app_tfidf, on='loan_id' , how='left')
print(var_app_tfidf.shape)
var_app_tfidf.to_csv(file_path + 'var_app_tfidf.csv', index = False)
var_dict_app_tfidf = dp.VarDict(var_app_tfidf.drop('loan_id', axis = 1), data_sorce='app_tfidf')
var_dict_app_tfidf.to_csv(os.path.join(file_path, 'var_dict_app_tfidf.csv'), index = False)
time2 = time.time()
print('run_time: ' ,time2-time1)

(3768952, 3)
(35959, 189)
run_time:  -12.010830640792847


dummy

In [4]:
time1 = time.time()
df_app = load_data_from_pickle(file_path,'df_app.pkl')
app_freq = pd.read_csv(file_path  + 'app_freq_train_dict.csv')
dict1 = dict(zip(app_freq['app'],app_freq['rate_freq']))
app_list = [k for k,v in dict1.items() if v>=0.1]
len(app_list)
df_app1 = df_app[df_app['appName'].isin(app_list)]
df_app1.shape
frame1 = pd.crosstab(df_app1['loan_id'],df_app1['appName'])
frame1.shape
save_data_to_pickle(frame1,file_path,'var_app.pkl')
frame1.head()
time2 = time.time()
print('run_time: ', time2 - time1)

222

(2201960, 2)

(35903, 222)

appName,AdaKami,AdaPundi,Agen Masukan Market,AgenMasukanMarket,Akulaku,Album,Analytics,Android Accessibility Suite,Android Easter Egg,Android System WebView,AndroidAccessibilitySuite,AndroidEasterEgg,AndroidSystemWebView,Atci_service,AudioEffect,BABE,BCA mobile,BCAmobile,BPJSTKU,Blibli.com,Briefing,Browser,Browserplug,Bukalapak,Cairin,Cashcepat,Cashwagon,Catatan,CatatanKeep,Chrome,ConfigUpdater,Cuaca,DANA,DanaRupiah,Download,Downloads,Drive,Duo,EasyShare,Easycash,Editorfoto,Email,Excel,Facebook,Facebook App Installer,Facebook App Manager,Facebook Services,FacebookAppInstaller,FacebookAppManager,FacebookServices,FileManager,Finmas,Foto,Foundation,FwkPlugin,GalaxyStore,GalaxyThemes,Galeri,GameLauncher,Gboard,GetApps,Gmail,Gojek,Google,Google Backup Transport,Google One Time Init,Google Play Film,Google Play Game,Google Play Musik,Google Play Store,Google Play services for Instant Apps,GoogleBackupTransport,GoogleOneTimeInit,GooglePlayFilm,GooglePlayGame,GooglePlayMusik,GooglePlayStore,GooglePlayservicesforInstantApps,Grab,Hangouts,Indodana,Instagram,JOOX,JULO,Jam,Ju0026T Express,KTA KILAT,KTAKILAT,Kalender,Kalkulator,Kamera,Kamus,Kerangka Kerja Layanan Google,KerangkaKerjaLayananGoogle,Kloningponsel,Kompas,KonsolFail,Kontak,Kredinesia,Kredit Pintar,KreditPintar,Kredivo,Kunci,KunciLayarSatuKetuk,LINE,Layanan Google Play,Layanan Transfer Backup Google,LayananGooglePlay,LayananTransferBackupGoogle,Lazada,Lens,LinkSharing,LinkedIn,Lite,MIUISDK,Maps,Mesin Google Text-to-speech,MesinGoogleText-to-speech,Mesinyangberagam,Messenger,MiVideo,MiWebView,MobileLegends:BangBang,Musik,MyHomeCreditIndonesia,MyTelkomsel,OVO,OneDrive,PartnerNetflixActivation,Pembayaranaman,Pembersih,Pemindai,Pencarian,PengelolaAkunGoogle,Penyiapan Android,Penyiapan Mitra Google,PenyiapanAndroid,PenyiapanMitraGoogle,Peramban,Perekam,PerekamSuara,PesanSistem,PhotoGrid,Photos,Pinjam Yuk,PinjamYuk,PinjamanGo,PowerPoint,Radio,RekamanLayar,Rupiah Cepat,RupiahCepat,SHAREit,SLIME,SamsungGiftIndonesia,SamsungInternet,SamsungMembers,SamsungNotes,SamsungPushService,ShareMe,Shelper,Shopee,Shopintar,Siaransel,Sinkronisasi Google Kalender,Sinkronisasi Google Kontak,SinkronisasiGoogleKalender,SinkronisasiGoogleKontak,Skype,Spotify,TalkBack,Telegram,Tema,TikTok,Tokopedia,TouchPal Indonesian Pack,TouchPal Pinyin/Bihua Pack,TouchPalArabicPack,TouchPalBengaliPack,TouchPalBopomofoPack,TouchPalBurmesePack,TouchPalEnglishgbPack,TouchPalFrenchPack,TouchPalHindiPack,TouchPalIndonesianPack,TouchPalMalayPack,TouchPalPinyin/BihuaPack,TouchPalRussianPack,TouchPalSinhalaPack,TouchPalSkinPackDefaultWhite,TouchPalSpanishPack,TouchPalTagalogPack,TouchPalTamilPack,TouchPalTeluguPack,TouchPalThaiPack,TouchPalUrduPack,TouchPalZAWGYIPack,Traveloka,TunaiCepat,TunaiKita,Tunaiku,Twitter,UCBrowser,UKU,UangMe,Unduhan,V-Appstore,Video(pemutar),WPS Office,WPSOffice,WearableManagerInstaller,WhatsApp,WizardPenyiapan,Word,Xiaomiserviceframework,YouTube,com.android.fileexplorer.FileExplorerApplication,com.mfashiongallery.emag,maucash,msa,org.codeaurora.bluetooth,tiket.com
loan_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
437422113627406336,1,1,0,1,0,0,1,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1,1,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,1,1,1,1,1,1,0,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,1,1,0,1,0,1,1,1,0,1,0,1,0,0,1,0,1,1,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,0,0,0,0,0,1,0,0,1,1,1,0,0,1,0,0
437483994752647168,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,1,1,0,1,1,0,1,1,0,0,1,1,1,1,0,0,0,1,1,1,0,0,1,1,0,1,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,1,1,0,1,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,0,1,0,0,0,0

run_time:  229.1925709247589
